# This Notebook has the following:

#### 1. Imports
#### 2. Useful helper functions/Examples/Experiments on Groove Midi Data
#### 3. Loading the official Groovae Dataset as a tfds dataset

## 1. Import Magenta and TF on local Machine

In [2]:
import time
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy
tf.enable_eager_execution()
import magenta.models
import magenta.music as mm
from magenta.music import midi_synth
import copy
import librosa

In [4]:
# import fluidsynth #Works from cmd using fluidsynth -g 1 -n -i PATH_to_SF2 PATH_TO_MIDI

#### 1.1 Import Config Classes(which are passed into the model during training)

In [2]:
import magenta.models.music_vae.configs as configs

#### 1.2 Import various experimental functions(Used Later)

In [25]:
from magenta.music.midi_io import midi_file_to_note_sequence

from magenta.pipelines.drum_pipelines import extract_drum_tracks

from magenta.music.drums_lib import DrumTrack

from magenta.music.sequences_lib import *
from magenta.music.sequences_lib import is_absolute_quantized_sequence
from magenta.music.sequences_lib import is_relative_quantized_sequence
from magenta.music.sequences_lib import quantize_note_sequence
from magenta.music.sequences_lib import quantize_note_sequence_absolute

#### 1.3 Define list of available drum pitch classes(Not used here)

In [1]:
# FULL_DRUM_PITCH_CLASSES = [
#     [p] for p in  # pylint:disable=g-complex-comprehension
#     [36, 35, 38, 27, 28, 31, 32, 33, 34, 37, 39, 40, 56, 65, 66, 75, 85, 42, 44,
#      54, 68, 69, 70, 71, 73, 78, 80, 46, 67, 72, 74, 79, 81, 45, 29, 41, 61, 64,
#      84, 48, 47, 60, 63, 77, 86, 87, 50, 30, 43, 62, 76, 83, 49, 55, 57, 58, 51,
#      52, 53, 59, 82]
# ]
# ROLAND_DRUM_PITCH_CLASSES = [
#     # kick drum
#     [36],
#     # snare drum
#     [38, 37, 40],
#     # closed hi-hat
#     [42, 22, 44],
#     # open hi-hat
#     [46, 26],
#     # low tom
#     [43, 58],
#     # mid tom
#     [47, 45],
#     # high tom
#     [50, 48],
#     # crash cymbal
#     [49, 52, 55, 57],
#     # ride cymbal
#     [51, 53, 59]
# ]

# # print(FULL_DRUM_PITCH_CLASSES)

## 2. Examples with MIDI data

In [6]:
path = 'C:\\Users\\Anirudh\\PycharmProjects\\Magenta\\Data\\groove\\drummer1\\session1\\1_funk_80_beat_4-4.mid' #path to a .mid on your local machine

test = midi_file_to_note_sequence(path) #Convert to a NoteSequence object

# print(test) #print the NoteSequence object
# dir(test) #Contents of the object
# print(test.quantization_info.steps_per_second) #Check the qunatization info of the object 

#### 2.1 Helper Functions from official Groovae notebook

In [22]:
#######################################################################################
#######################################################################################
########## UTILS GrooVAE: Generating and Controlling Expressive Drum Performances #####
###################### BY Jon Gillick, Adam Roberts, Jesse Engel ######################
#######################################################################################

#link to the GROOVAE notebook: https://goo.gl/magenta/groovae-colab

 
################################# Load some configs to be used later ##################

dc_quantize = configs.CONFIG_MAP['groovae_2bar_humanize'].data_converter
dc_tap = configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity'].data_converter
dc_hihat = configs.CONFIG_MAP['groovae_2bar_add_closed_hh'].data_converter
dc_4bar = configs.CONFIG_MAP['groovae_4bar'].data_converter

########################################################################################



# Define some functions

# If a sequence has notes at time before 0.0, scootch them up to 0
def start_notes_at_0(s):
  for n in s.notes:
    if n.start_time < 0:
      n.end_time -= n.start_time
      n.start_time = 0
  return s

#requires installation of pyfluidsynth(there are issues on windows)
def play(note_sequence, sf2_path='Standard_Drum_Kit.sf2'):  
  if sf2_path:
    audio_seq = midi_synth.fluidsynth(start_notes_at_0(note_sequence), sample_rate=44100, sf2_path=sf2_path)
    IPython.display.display(IPython.display.Audio(audio_seq, rate=44100))
  else:
    mm.play_sequence(start_notes_at_0(note_sequence), synth=mm.fluidsynth)

# Some midi files come by default from different instrument channels
# Quick and dirty way to set midi files to be recognized as drums

def set_to_drums(ns):
  for n in ns.notes:
    n.instrument=9
    n.is_drum = True
    
def unset_to_drums(ns):
  for note in ns.notes:
    note.is_drum=False
    note.instrument=0
  return ns

# quickly change the tempo of a midi sequence and adjust all notes
def change_tempo(note_sequence, new_tempo):
  new_sequence = copy.deepcopy(note_sequence)
  ratio = note_sequence.tempos[0].qpm / new_tempo
  for note in new_sequence.notes:
    note.start_time = note.start_time * ratio
    note.end_time = note.end_time * ratio
  new_sequence.tempos[0].qpm = new_tempo
  return new_sequence

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

def download_audio(audio_sequence, filename, sr):
  librosa.output.write_wav(filename, audio_sequence, sr=sr, norm=True)
  files.download(filename)

# quick method for removing microtiming and velocity from a sequence
def get_quantized_2bar(s, velocity=0):
  new_s = dc_quantize.to_notesequences(dc_quantize.to_tensors(s).inputs)[0]
  new_s = change_tempo(new_s, s.tempos[0].qpm)
  if velocity != 0:
    for n in new_s.notes:
      n.velocity = velocity
  return new_s

# quick method for turning a drumbeat into a tapped rhythm
def get_tapped_2bar(s, velocity=85, ride=False):
  new_s = dc_tap.to_notesequences(dc_tap.to_tensors(s).inputs)[0]
  new_s = change_tempo(new_s, s.tempos[0].qpm)
  if velocity != 0:
    for n in new_s.notes:
      n.velocity = velocity
  if ride:
    for n in new_s.notes:
      n.pitch = 42
  return new_s

# quick method for removing hi-hats from a sequence
def get_hh_2bar(s):
  new_s = dc_hihat.to_notesequences(dc_hihat.to_tensors(s).inputs)[0]
  new_s = change_tempo(new_s, s.tempos[0].qpm)
  return new_s


# Calculate quantization steps but do not remove microtiming
def quantize(s, steps_per_quarter=4):
  return mm.sequences_lib.quantize_note_sequence(s,steps_per_quarter)

# Destructively quantize a midi sequence
def flatten_quantization(s):
  beat_length = 60. / s.tempos[0].qpm
  step_length = beat_length / 4#s.quantization_info.steps_per_quarter
  new_s = copy.deepcopy(s)
  for note in new_s.notes:
    note.start_time = step_length * note.quantized_start_step
    note.end_time = step_length * note.quantized_end_step
  return new_s

# Calculate how far off the beat a note is
def get_offset(s, note_index):
  q_s = flatten_quantization(quantize(s))
  true_onset = s.notes[note_index].start_time
# print(true_onset)
  quantized_onset = q_s.notes[note_index].start_time
  diff = quantized_onset - true_onset
# print(diff)
  beat_length = 60. / s.tempos[0].qpm
# print(beat_length)
  step_length = beat_length / 4#q_s.quantization_info.steps_per_quarter
  offset = diff/step_length
  return offset

def is_4_4(s):
  ts = s.time_signatures[0]
  return (ts.numerator == 4 and ts.denominator ==4)


def preprocess_4bar(s):
  return dc_4bar.to_notesequences(dc_4bar.to_tensors(s).outputs)[0]


def preprocess_2bar(s):
  return dc_quantize.to_notesequences(dc_quantize.to_tensors(s).outputs)[0]

def _slerp(p0, p1, t):
  """Spherical linear interpolation."""
  omega = np.arccos(np.dot(np.squeeze(p0/np.linalg.norm(p0)),
    np.squeeze(p1/np.linalg.norm(p1))))
  so = np.sin(omega)
  return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1


#### 2.2 Play NoteSequence(open issues on Github)

In [21]:
# !pip install pyfluidsynth #pyfluidsynth is required to play a NoteSequenece object

# Facing unresolved issues on Windows

# play(test,sf2_path='Standard_Drum_Kit.sf2')

#### 2.3 To check if obtained NoteSequence object is quantized
#### There are two types of Quantization in the NoteSequence Object:
* Steps per quarter
* Absolute time


In [28]:
print(is_absolute_quantized_sequence(test)) #Checks if note_sequence.quantization_info.steps_per_second > 0
print(is_relative_quantized_sequence(test)) #Checks if note_sequence.quantization_info.steps_per_quarter > 0

#Hence we know that our midi file is not Quantized w.r.t either parameters

False
False


#### 2.4 NoteSequence can be quantized as follows:

In [40]:
#Quantized using steps per Quarter
quantized_mid = quantize(test) #Using the Quantize function from 2.1
print(is_relative_quantized_sequence(quantized_mid))
print(is_absolute_quantized_sequence(quantized_mid))

#Quantization by absolute time
quantized_mid_2 = quantize_note_sequence_absolute(test,2560) #second argument is steps_per_second(Each second will be divided into this many quantized timesteps.)
print(is_relative_quantized_sequence(quantized_mid_2))
print(is_absolute_quantized_sequence(quantized_mid_2))

# print(quantized_mid)
# print(quantized_mid_2)

# print(quantized_mid.quantization_info)
# print(quantized_mid_2.quantization_info)

True
False
False
True


#### 2.5 Experiments with extract_drum_tracks function from 'magenta.pipelines.drum_pipelines'
Given Function Documentation

"""Extracts a list of drum tracks from the given quantized NoteSequence.

  This function will search through `quantized_sequence` for drum tracks. A drum
  track can span multiple "tracks" in the sequence. Only one drum track can be
  active at a given time, but multiple drum tracks can be extracted from the
  sequence if gaps are present.

  Once a note-on drum event is encountered, a drum track begins. Gaps of silence
  will be splitting points that divide the sequence into separate drum tracks.
  The minimum size of these gaps are given in `gap_bars`. The size of a bar
  (measure) of music in time steps is computed form the time signature stored in
  `quantized_sequence`.

  A drum track is only used if it is at least `min_bars` bars long.

  After scanning the quantized NoteSequence, a list of all extracted DrumTrack
  objects is returned.

  Args:
    quantized_sequence: A quantized NoteSequence.
    search_start_step: Start searching for drums at this time step. Assumed to
        be the beginning of a bar.
    min_bars: Minimum length of drum tracks in number of bars. Shorter drum
        tracks are discarded.
    max_steps_truncate: Maximum number of steps in extracted drum tracks. If
        defined, longer drum tracks are truncated to this threshold. If pad_end
        is also True, drum tracks will be truncated to the end of the last bar
        below this threshold.
    max_steps_discard: Maximum number of steps in extracted drum tracks. If
        defined, longer drum tracks are discarded.
    gap_bars: A drum track comes to an end when this number of bars (measures)
        of no drums is encountered.
    pad_end: If True, the end of the drum track will be padded with empty events
        so that it will end at a bar boundary.
    ignore_is_drum: Whether accept notes where `is_drum` is False.

  Returns:
    drum_tracks: A python list of DrumTrack instances.
    stats: A dictionary mapping string names to `statistics.Statistic` objects.

  Raises:
    NonIntegerStepsPerBarError: If `quantized_sequence`'s bar length
        (derived from its time signature) is not an integer number of time
        steps.
  """

In [42]:
extracted_drum_tracks = extract_drum_tracks(quantized_mid)
drum_tracks , stats = extracted_drum_tracks
print(drum_tracks)
print(stats)

dict_values([<magenta.pipelines.statistics.Counter object at 0x000001FAD7F4D6C8>, <magenta.pipelines.statistics.Counter object at 0x000001FAD7F4D748>, <magenta.pipelines.statistics.Counter object at 0x000001FAD7F4D808>, <magenta.pipelines.statistics.Histogram object at 0x000001FAD7F4D608>])


* This object is instantiated as an attribute for DrumsConverter Class in "magenta.models.music_vae.data"
* Later a DrumsConverter Object is passed into a Config Object in "magenta.models.music_vae.configs"
* Further this Config object is passed into training by calling the model.

#### 2.6 Using config Objects to(using functions in 2.1):
* remove microtiming and velocity
* change drumbeat into a tapped rhythm
* removing hi-hats

In [46]:
stripped1 = get_quantized_2bar(test,velocity = 0)
stripped2 = get_tapped_2bar(test,velocity = 80, ride = False)
stripped3 = get_hh_2bar(test)
# print(stripped)
# print(stripped2)
# print(stripped3)

#### 2.7 Check offsets from defined grid(steps per quarter)

In [48]:
print(get_offset(test,4))

print(get_offset(stripped1,4)) #stripped1 has been quantized_2bar

-0.01666666666666809
0.0


#### 2.8 Preprocess midi file using defind Configs
(using a function defined in 2.1)

In [79]:
preprocess_var = preprocess_4bar(test)
print(preprocess_var)

tempos {
  qpm: 120.0
}
notes {
  pitch: 38
  velocity: 7
  start_time: 1.4104166666666667
  end_time: 1.5354166666666667
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 10
  start_time: 1.4510416666666668
  end_time: 1.5760416666666668
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 56
  start_time: 1.6364583333333336
  end_time: 1.7614583333333336
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 48
  start_time: 1.7520833333333334
  end_time: 1.8770833333333334
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 65
  start_time: 1.7427083333333335
  end_time: 1.8677083333333335
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 47
  start_time: 1.8833333333333335
  end_time: 2.0083333333333337
  instrument: 9
  is_drum: true
}
notes {
  pitch: 36
  velocity: 42
  start_time: 1.9989583333333334
  end_time: 2.1239583333333334
  instrument: 9
  is_drum: true
}
notes {
  pitch: 46
  velocity: 48
  start_tim

#### 2.9 Destructively Qunatize midi file(using function from 2.1)

In [80]:
destructive_qunat = flatten_quantization(test)
print(destructive_qunat)

ticks_per_quarter: 480
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 80.0
}
notes {
  pitch: 38
  velocity: 7
  start_time: 39.75
  end_time: 40.875
  instrument: 9
  is_drum: true
  quantized_start_step: 212
  quantized_end_step: 218
}
notes {
  pitch: 38
  velocity: 10
  start_time: 40.875
  end_time: 41.625
  instrument: 9
  is_drum: true
  quantized_start_step: 218
  quantized_end_step: 222
}
notes {
  pitch: 38
  velocity: 56
  start_time: 45.9375
  end_time: 48.0
  instrument: 9
  is_drum: true
  quantized_start_step: 245
  quantized_end_step: 256
}
notes {
  pitch: 44
  velocity: 65
  start_time: 48.9375
  end_time: 51.0
  instrument: 9
  is_drum: true
  quantized_start_step: 261
  quantized_end_step: 272
}
notes {
  pitch: 38
  velocity: 48
  start_time: 49.3125
  end_time: 51.1875
  instrument: 9
  is_drum: true
  quantized_start_step: 263
  quantized_end_step: 273
}
notes {
  pitch: 38
  velocity: 47
  start_time: 53.0625
  end_time: 5

## 3. Trivially load the data Using TFDS

In [50]:
dataset_2bar = tfds.as_numpy(tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.VALIDATION))

In [123]:
##Features of the above dataset from the official TFDS Documentation

#link: https://www.tensorflow.org/datasets/catalog/groove

# FeaturesDict = {}

# FeaturesDict({
#     'bpm': tf.int32,
#     'drummer': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
#     'id': tf.string,
#     'midi': tf.string,
#     'style': FeaturesDict({
#         'primary': ClassLabel(shape=(), dtype=tf.int64, num_classes=18),
#         'secondary': tf.string,
#     }),
#     'time_signature': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
#     'type': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
# })


In [17]:
for ex in dataset_2bar:
    print(ex)

{'bpm': 180, 'drummer': 5, 'id': b'drummer6/session3/5:000:001:002:003:004:005:006:007:008:009:010:011:012:013', 'midi': b"MThd\x00\x00\x00\x06\x00\x01\x00\x02\x01\xe0MTrk\x00\x00\x00\x19\x00\xffQ\x03\x05\x16\x15\x00\xffX\x04\x04\x02\x18\x08\x00\xffY\x02\x00\x00\x01\xff/\x00MTrk\x00\x00\x00\xe0\x00\xff\x03\nMidi Drums\x00\xc9\x00\x00\xb9\x04$\x1e\x04\x1f+\x04\x1a6\x04\x15@\x04\x10\x14\x99$\x7f,\xb9\x04\x0ce\x99$\x00\x81\x067\x7f\x04&\x7f\x81\x0c7\x00\x05&\x00\x08\xb9\x04\x07\x81P\x04\x0bd\x99$\x7f\x1a\xb9\x04\x0c\x01\x99\x1a\x7fl\xb9\x04\x07\x0b\x99$\x00\x1a\x1a\x00\x82%(\x7f\r\xb9\x04\x04\x00\x99\x1a\x7f\x81\x05(\x00\x0e\x1a\x00\\$\x7f\x81\x12$\x00o\xb9\x04\x04\x00\x99\x1a\x7f\x81\x12\x1a\x00'$\x7fJ\xb9\x04\x00H\x99$\x00|(\x7f\x0f\xb9\x04\x02\x00\x99\x1a\x7fM\xb9\x04\x005\x99(\x00\x10\x1a\x00\x81?\xb9\x04\x04M\x04\tD\x04\t\x00\x99\x1a\x7f\x11$\x7fT\xb9\x04\x04-\x99\x1a\x00\x10$\x00\x825\xb9\x04\x04\x00\x99\x1a\x7f\t(|\x81\t\x1a\x00\x07(\x00S$\x7f\x81\x11$\x00b\xb9\x04\x07\x01\xff/\x00

{'bpm': 117, 'drummer': 0, 'id': b'drummer1/session3/15:000:001:002:003:004:005:006:007:008:009:010:011', 'midi': b'MThd\x00\x00\x00\x06\x00\x01\x00\x02\x01\xe0MTrk\x00\x00\x00\x19\x00\xffQ\x03\x07\xd34\x00\xffX\x04\x04\x02\x18\x08\x00\xffY\x02\x00\x00\x01\xff/\x00MTrk\x00\x00\x01\x81\x00\xff\x03\nMidi Drums\x00\xc9\x00\x00\xb9\x04Z\x7f\x04Z\x00\x99*\x19_*\x00\x10\xb9\x04Z\x00\x99*\x1f_*\x00\x15\xb9\x04Z\x00\x99*&_*\x00%\xb9\x04Z\x00\x99*m^*\x00\x13\xb9\x04Z\x00\x99*\x16_*\x00)\xb9\x04Z\x00\x99*\x08_*\x00\x0f\xb9\x04Z\x00\x99*\x1c^*\x00\x1e\xb9\x04Z\x00\x99*~_*\x00!\xb9\x04Z\x00\x99*!^*\x00\x14\xb9\x04Z\x00\x99*\x0f^*\x00\x15\xb9\x04Z\x00\x99*\x1b_*\x00&\xb9\x04Z\x00\x99*]_*\x00\x15\xb9\x04Z\x00\x99*\x1aU\xb9\x04:\n\x99*\x00\x0f\x1a[\x05\xb9\x04\x00Z\x99\x1a\x00>\xb9\x04\x00\x00\x99.\x07C\xb9\x04M\x00\x99,<\x14*M\x07.\x00\x03\xb9\x04ZA\x99,\x00\x13*\x00\x13\xb9\x04Z\x00\x99*\x11_*\x00\'\xb9\x04Z\x00\x99* ^*\x00\r\xb9\x04Z\x00\x99*)_*\x00\x1c\xb9\x04Z\x00\x99*e_*\x00\x1c\xb9\x04Z\x00\x9

{'bpm': 89, 'drummer': 4, 'id': b'drummer5/session1/4:000', 'midi': b'MThd\x00\x00\x00\x06\x00\x01\x00\x02\x01\xe0MTrk\x00\x00\x00\x19\x00\xffQ\x03\nIm\x00\xffX\x04\x04\x02\x18\x08\x00\xffY\x02\x00\x00\x01\xff/\x00MTrk\x00\x00\x02\x0c\x00\xff\x03\nMidi Drums\x00\xc9\x00\r\x99&7\x08$(@&\x00\x06&\'\x01$\x00;&\x00\x00&\'\n\xb9\x04\x00\x02\x99&\x008&.\x04&\x00\x07\xb9\x04 \x08\x99,(\x16\xb9\x04Z2\x99,\x00\x18%A\x12$!6%\x00\x12$\x00\x17&#@&\x00\x00&\x1e\x08&\x00\x1d&*#&\x00\x16\xb9\x04\x00\x00\x99&,\x0e&\x00F\xb9\x04!\x00\x99&G\n$\x1c\x00, \x16\xb9\x04Z(\x99&\x00\t$\x00\x00,\x00\x1e&6G&\x006&D\x17$.1&\x00\x17$\x00\t&&\x11\xb9\x04J\x1f\x04\x00\x18\x99&\x006\xb9\x04 \x04\x99&4\x05,#\x17\xb9\x04Z,\x99&\x00\x05,\x00*&A\r$\x14;&\x00\r$\x00;+YH+\x00+&#\x05\xb9\x04J \x04\x00#\x99&\x00\x18&\'\x15\xb9\x04\x10\n\x99,2\x16\xb9\x04Z\x13\x99&\x00\x1f,\x00\x1b&KH&\x001&. $$\x1d&\x00\x00&1\x0b&\x00 $\x00\t&0\x14&\x00\x07\xb9\x04K\x1b\x99&/\t\xb9\x04\x00\x08\x99&\x00?&;\x06\xb9\x04 \x08\x99,/\x16\xb9\x04Z$

{'bpm': 115, 'drummer': 0, 'id': b'drummer1/session3/13:000:001:002:003:004:005:006:007:008:009:010:011:012:013:014:015:016:017:018:019:020:021:022:023:024:025:026:027:028:029:030:031:032:033:034:035:036:037:038:039:040:041:042:043:044:045:046:047:048:049:050:051:052:053:054:055:056:057:058:059:060:061:062:063:064:065:066:067:068:069:070:071:072:073:074:075:076:077:078:079:080:081:082:083:084:085:086:087:088:089:090:091:092:093:094:095:096:097:098:099:100:101:102:103:104', 'midi': b'MThd\x00\x00\x00\x06\x00\x01\x00\x02\x01\xe0MTrk\x00\x00\x00\x19\x00\xffQ\x03\x07\xf6\x0b\x00\xffX\x04\x04\x02\x18\x08\x00\xffY\x02\x00\x00\x01\xff/\x00MTrk\x00\x00\x02r\x00\xff\x03\nMidi Drums\x00\xc9\x00\x00\xb9\x04D\x08\x99+N\x02$\x7f\x07\xb9\x04ZU\x99+\x00\x01$\x00\x11\xb9\x04D(\x04"+\x04@$\x04Z\x0b\x99+X]+\x00\x07\xb9\x04*\x1e\x04\x00W\x04B\x03\x99,F\x13(\x7f\x00+f\t\xb9\x04Z@\x99,\x00\x14(\x00\x00+\x00\x15\xb9\x041*\x04\x14$\x04E\x02\x99,%\x1d\xb9\x04Z\x0c\x99$:\x04+T0,\x00$\xb9\x04*\x05\x99$\x00\x03+

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [51]:
dev_sequences = [quantize(mm.midi_to_note_sequence(features["midi"])) for features in dataset_2bar]
_ = [set_to_drums(s) for s in dev_sequences]
dev_sequences = [s for s in dev_sequences if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > mm.steps_per_bar_in_quantized_sequence(s)]
